In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)


In [6]:
scales = [4]
# to conform to traditional regression, here only one column of x
selected_columns = ['x1', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.linear_regression(x, y)

error = 0.05
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

result:  tensor([[[-0.0368],
         [14.5325]]])


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:172: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:399: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  y = torch.where(y==MagicNumber, torch.tensor(0.0), y)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:401: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if yo

In [8]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.
Using 2 columns for non-linearity table.


result:  tensor([[[1.5701],
         [9.5437]]])
==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 4, param_scale: 4, scale_input_multiplier: 1) ------------->

+---------------+--------------+--------------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error    | median_error | max_error    | min_error    | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+---------------+--------------+--------------+--------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| -0.0037356615 | 0.0076099634 | 0.0076099634 | -0.018816948 | 0.00880897     | 0.0076099634     | 0.018816948   | 0             | 0.00013732968      | 0.0009583685       | 0.002272812            |
+---------------+--------------+--------------+--------------+----------------+---------

scale:  [4]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":4,"param_scale":4,"scale_rebase_multiplier":1,"lookup_range":[-13092,184230],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":1518,"total_const_size":219,"model_instance_shapes":[[1],[1,2,1]],"model_output_scales":[0,4],"model_input_scales":[4,4],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[[-8,8]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709689578404}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 27.76781916618347 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 1.5625
witness result 2 : 9.5625
==== Generating Proof ====
proof:  {'instances': [['78f113a919d1324cbee267b4320db42ee0170745a57013fa302f8139d7c36f18', '8dff683cfffbd97a4d94de7d897e03bb3f2fa18084beff4bbfbd152c2e2bcd16', '0100000000000000000000000000000000000000000000000000000000000000', '1900000000000000000000000000000000000000000000000000000000000000', '9900000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x26531800aa72b5c475b89752ec218e3c3540a72c934d711e5fddc08c39ec9e7500920f846837e5d6ccfd4ce080271d118b5baef638009e8338cf4fafaffc30cb1c4df8aface62a075d51de1f421e963266ac630ac426f07714c727f10dad324d2e7b62326a1db2fc495c2860da2c42556e66ad7b819d67fbadb1e1af00584250212dab3be30c803dcd884f0ea319bdf08613464d70733d76b61b4d4ba7adb014106fe882cbb3b12f28c090a98fee91a3563c602f64cdfecc6191561cb696439b1d8d99223d2fcaeae42fb701bc5734a30e84fb868ee3

In [10]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [1.5625, 9.5625]
